# Plot Source Time in Selected Objects

 work with Weakly_2023_44
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/30
- update : 2023/12/30


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (16,10)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
# A repo which exists at the USDF, contains data from various instruments and
REPO = '/repo/embargo'

from lsst.daf.butler import Butler
from pprint import pprint

# Here we specify a default collection to use
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
butler = Butler(REPO, collections=my_collection)
registry = butler.registry

In [ ]:
#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle
import lsst.sphgeom

import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [ ]:
Angle(3.1415).asDegrees()

In [ ]:
Angle(3.1415).asRadians()

In [ ]:
SpherePoint(Angle(3.1415),Angle(1.0))

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates

In [ ]:
import pandas as pd

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Config

### Galactic Plane

In [ ]:
gal_long = np.linspace(-180.,180,360)
gal_lat = np.zeros((360))

coordinates_galactic_planes = SkyCoord(l=gal_long*u.degree, b=gal_lat*u.degree, frame='galactic')
gp_radec = coordinates_galactic_planes.transform_to('icrs')  

gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)

### Sources

In [ ]:
file = "sourceTable_202312.csv"
file_visit = "sourceTablevisit_202312.csv"

In [ ]:
suptitle = "Sources, Auxtel phot 2023"

## Skymap, Tracts and Patchs

In [ ]:
Auxtel2023_tracts = [ 5616,5634,5615,10644,383,8188,3863,455,452,5839,10643,382]

In [ ]:
skymap_auxtel ='latiss_v1'

In [ ]:
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=my_collection)

In [ ]:
with open('Latiss_tractspatchs.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

### Should try Tracts
- 5615 - OK , Not in our galaxy
- 5616 - Non, 1 nuit
- 5839 - Non, 1 nuit
- 5634 - OK - Galaxie Plane

In [ ]:
tract_selected = 5615

In [ ]:
tractInfo = skymap.generateTract(tract_selected)

In [ ]:
tractInfo.contains 

In [ ]:
def SourceInTract(row):
    #coord_dec
    #coord_ra
    #print(row)
    try:
        ra = row["ra"]
        dec = row["dec"]
        ra_rad = ra*np.pi/180.
        dec_rad = dec*np.pi/180.
        spherepoint = SpherePoint(Angle(ra_rad),Angle(dec_rad))
        flag = tractInfo.contains( spherepoint)
        return flag
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        return False
        
    

### Selected Objects

In [ ]:
fileobjects_in = "objectTableselected_202312.csv"

In [ ]:
df_selobj = pd.read_csv(fileobjects_in)

In [ ]:
index_selected_object = 10
the_obj=df_selobj.iloc[index_selected_object]

In [ ]:
the_obj

In [ ]:
suptitle += f" {index_selected_object}) Object {the_obj.objectId }"

In [ ]:
the_obj.objectId 

In [ ]:
vect_to_object = lsst.sphgeom.UnitVector3d(lsst.sphgeom.LonLat.fromDegrees(the_obj.coord_ra, the_obj.coord_dec))

In [ ]:
spherePointCenter = lsst.geom.SpherePoint(the_obj.coord_ra*lsst.geom.degrees, the_obj.coord_dec*lsst.geom.degrees)

## Input

In [ ]:
df = pd.read_csv(file,index_col=0)
#df_visit = pd.read_csv(file_visit,index_col=0)

In [ ]:
list_of_columns = sorted(list(df.columns))

In [ ]:
#for col in list_of_columns:
#    print(col)

## Selection

In [ ]:
df.dropna(inplace=True)
selection1 = df["extendedness_flag"] == False
selection2 = df["detect_isPrimary"] == True
df = df[selection1 & selection2 ]

### Selection for one object

In [ ]:
#df["inptract"]=df[["ra","dec"]].apply(SourceInTract)

In [ ]:
cut_arcsc = 12.0

In [ ]:
all_angles = []
for index, row in df.iterrows():
    spt = lsst.geom.SpherePoint(row["coord_ra"]*lsst.geom.degrees, row["coord_dec"]*lsst.geom.degrees)
    ang = spherePointCenter.separation(spt)
    ang_arcsec = ang.asArcseconds()
    all_angles.append(ang_arcsec)

In [ ]:
df["sep"] = all_angles

In [ ]:
df["sep"].min() 

In [ ]:
df.sort_values(by=["dateobs"],inplace=True)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,4))
df['sep'].plot(kind="hist",bins=50,range=(0,50),ax=ax,grid=True,title="separation angle")
ax.axvline(cut_arcsc,color="k")
ax.set_xlabel("$\\theta$ (arcsec)")
plt.suptitle(suptitle)

In [ ]:
selection3 = df['sep']< cut_arcsc
df = df[selection3 ]

## Magnitudes

In [ ]:
df["psfMag"] = df["psfFlux"].map(convert_fluxtomag)
df["gaussianMag"] = df["gaussianFlux"].map(convert_fluxtomag)
df["calibMag"] = df["calibFlux"].map(convert_fluxtomag)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))

df['psfMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="psfMag",ax=ax1)
df['gaussianMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="gaussianMag",ax=ax2)
df['calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="grey",logy=True,grid=True,title="calibMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()


## Convert dateobs into datetime

In [ ]:
df[["dateobs","sep","band","ccdVisitId",'x','y']]

In [ ]:
df["Time"] = pd.to_datetime(df['dateobs'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,4))
df.plot(x="Time",y="ccdVisitId",marker='o',c="r",lw=0.0,ax=ax,grid=True)
plt.suptitle(suptitle)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1)

df[df["band"]=="g"].plot(x="Time",y="psfMag",marker='o',c="g",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="psfMag",marker='o',c="r",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="psfMag",marker='o',c="y",lw=0.0,ax=ax1,grid=True,legend=False)
ax1.set_title("psfMag")

df[df["band"]=="g"].plot(x="Time",y="gaussianMag",marker='o',c="g",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="gaussianMag",marker='o',c="r",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="gaussianMag",marker='o',c="y",lw=0.0,ax=ax2,grid=True,legend=False)
ax2.set_title("gaussianMag")

df[df["band"]=="g"].plot(x="Time",y="calibMag",marker='o',c="g",lw=0.0,ax=ax3,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="calibMag",marker='o',c="r",lw=0.0,ax=ax3,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="calibMag",marker='o',c="y",lw=0.0,ax=ax3,grid=True,legend=False)
ax3.set_title("calibMag")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,5))
df_sel = df["coord_ra"]
df_sel.plot(kind="hist",x="coord_ra",bins=45,range=(0,360),facecolor="b",ax=ax1,grid=True,logy=True)
df_sel = df["coord_dec"]
df_sel.plot(kind="hist",x="coord_dec",bins=55,range=(-90,15),facecolor="g",ax=ax2,grid=True,logy=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
list_of_ccdvisits = df[["ccdVisitId"]].values.flatten()
list_of_ccdvisits

In [ ]:
dataId = {'instrument': 'LATISS', 'detector': 0}
df_visit = butler.get('ccdVisitTable', dataId=dataId)

In [ ]:
df_visit.columns

In [ ]:
df_visit.loc[list_of_ccdvisits[0]]

In [ ]:
#print(list(df_visit.index))

In [ ]:
#results = set(butler.registry.queryDataIds('visit', dataId={"tract":tract_selected, "skymap":"latiss_v1"}))
#pprint(results)

In [ ]:
datasettype = 'calexp.visitInfo'
all_calexp = []
all_dataId = []
for idx,ccdvisit in enumerate(list_of_ccdvisits):
    try:
        visitid = df_visit.loc[ccdvisit]["visitId"]
        dayobs = visitid //100_000
        dataId = {'visit': visitid, 'detector': 0}
        visitInfo = butler.get(datasettype, dataId=dataId )
        calexp = butler.get('calexp', dataId=dataId)
        if idx<2:
            all_calexp.append(calexp)
            all_dataId.append(dataId)
        #print(visitInfo)
        print(visitid,dayobs,":","az=",visitInfo.boresightAzAlt[0],"alt=",visitInfo.boresightAzAlt[1],"am=",visitInfo.boresightAirmass)
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(all_calexp[0].image,title=suptitle)

with afw_display.Buffering():
    x = df.iloc[0]['x']
    y = df.iloc[0]['y']
    afw_display.dot('o', x , y, size=100, ctype='orange')
    afw_display.dot('+', x , y, size=10, ctype='red')

plt.gca().axis('on')
